## Before this notebook can be used to run NSG-R jobs:

   - The users must register at NSG Portal (https://www.nsgportal.org/reg/reg.php) and enter their credentials here.
   - An Application Key must be obtained:
       - log in at (https://nsgr.sdsc.edu:8443/restusers/login.action)
       - create a new application at Developer->Application Management (Create New Application)
       - save the Application Key for use in REST requests and enter it here when prompted
   - A jupyter notebook server must be installed to run the notebook
    

In [1]:
# This worked with python 3, with requests module installed
# use port 8443 for production, 8444 for test
# register at https://www.nsgportal.org/reg/reg.php for username and password
CRA_USER = input("Enter your user name ")

from getpass import getpass
PASSWORD = getpass(prompt='Enter your password: ')
# for development version:
# log in at https://nsgr.sdsc.edu:8444/restusers/login.action
# for production version:
# log in at https://nsgr.sdsc.edu:8443/restusers/login.action
# Tool names can be found at Developer->Documentation (Tools: How to Configure Specific Tools)
# save the Application Key for use in REST requests
KEY = input("Enter your application key ")
# for development version:
# URL = 'https://nsgr.sdsc.edu:8444/cipresrest/v1'
# for production version:
URL = 'https://nsgr.sdsc.edu:8443/cipresrest/v1'
TOOL = 'NEURON73_TG'


Enter your user name aacwb
Enter your password: ········
Enter your application key My_second_app-9DDF36CF85ED464B979449B36BDA8B33


## Run the below cell to select the input folder

In [2]:
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
import os

class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook



SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

## Run this cell to submit the Job

In [3]:



%%capture --no-stderr submitoutput

import os
import requests
import xml.etree.ElementTree
import time
import sys

a = my_button.files
folder_selected = a[0]


os.popen('pwd').read()
os.popen('ls -ltr').read()
headers = {'cipres-appkey' : KEY}
payload = {'tool' : TOOL, 'metadata.statusEmail' : 'true', 'vparam.number_cores_' : 24, 'vparam.number_nodes_' : 2, 'vparam.runtime_' : 0.5, 'vparam.filename_': 'Batch.hoc', 'vparam.cmdlineopts_': '-c TSTOP=1'}
files = {'input.infile_' : open(folder_selected,'rb')}

r = requests.post('{}/job/{}'.format(URL, CRA_USER), auth=(CRA_USER, PASSWORD), data=payload, headers=headers, files=files)
#print(r.text)

root = xml.etree.ElementTree.fromstring(r.text)

sys.stderr.write("%s\n" % r.text)
sys.stderr.write("%s\n" % r.url)

for child in root:
    if child.tag == 'resultsUri':
        for urlchild in child:
            if urlchild.tag == 'url':
                outputuri = urlchild.text
    if child.tag == 'selfUri':
        for urlchild in child:
            if urlchild.tag == 'url':
                selfuri = urlchild.text
#print(outputuri,file=sys.stderr)
sys.stderr.write("%s\n" % outputuri)
#print(selfuri,file=sys.stderr)
sys.stderr.write("%s\n" % selfuri)

headers = {'cipres-appkey' : KEY}

#print('Waiting for job to complete',file=sys.stderr)
sys.stderr.write('Waiting for job to complete\n')
jobdone = 0
while jobdone == 0:
    r = requests.get(selfuri, auth=(CRA_USER, PASSWORD), headers=headers)
    #print(r.text)
    root = xml.etree.ElementTree.fromstring(r.text)
    for child in root:
        if child.tag == 'terminalStage':
            jobstatus = child.text
            if jobstatus == 'false':
                time.sleep(5)
                #print('.',file=sys.stderr,end='')
                sys.stderr.write('.')
            else:
                jobdone = 1
                #print('',file=sys.stderr,end='\n')
                sys.stderr.write('\n')
                break

#print('Job completion detected, getting download URIs...',file=sys.stderr)
sys.stderr.write('Job completion detected, getting download URIs...')
r = requests.get(outputuri,
                 headers= headers, auth=(CRA_USER, PASSWORD))
#print(r.text)
globaldownloadurilist = []
root = xml.etree.ElementTree.fromstring(r.text)
for child in root:
    if child.tag == 'jobfiles':
        for jobchild in child:
            if jobchild.tag == 'jobfile':
                for downloadchild in jobchild:
                    if downloadchild.tag == 'downloadUri':
                        for attchild in downloadchild:
                            if attchild.tag == 'url':
                                #print(attchild.text)
                                sys.stdout.write(attchild.text)
                                globaldownloadurilist.append(attchild.text)
                
#print('Download complete.  Run the next cell.',file=sys.stderr)
sys.stderr.write('Download complete.  Run the next cell.\n')

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<jobstatus>
    <selfUri>
        <url>https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-2B2BF7E2334B4D2987614C78545DC413</url>
        <rel>jobstatus</rel>
        <title>NGBW-JOB-NEURON73_TG-2B2BF7E2334B4D2987614C78545DC413</title>
    </selfUri>
    <jobHandle>NGBW-JOB-NEURON73_TG-2B2BF7E2334B4D2987614C78545DC413</jobHandle>
    <jobStage>QUEUE</jobStage>
    <terminalStage>false</terminalStage>
    <failed>false</failed>
    <metadata>
        <entry>
            <key>statusEmail</key>
            <value>true</value>
        </entry>
    </metadata>
    <dateSubmitted>2018-09-27T21:10:57-07:00</dateSubmitted>
    <resultsUri>
        <url>https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-2B2BF7E2334B4D2987614C78545DC413/output</url>
        <rel>results</rel>
        <title>Job Results</title>
    </resultsUri>
    <workingDirUri>
        <url>https://nsgr.sdsc.edu:8443/cipresrest/v1

## Run this cell to download the output files

In [4]:
# download all output files

import re
#downloaduri = 'https://cipresrest.sdsc.edu/cipresrest/v1/job/kenneth/NGBW-JOB-RAXMLHPC8_REST_XSEDE-EF0E26B61D2E4C0C8D02499D8068D873/output/11496'
for downloaduri in globaldownloadurilist:
    r = requests.get(downloaduri, auth=(CRA_USER, PASSWORD), headers=headers)
    #sys.stderr.write("%s\n" % r.json())
    #r.content
    d = r.headers['content-disposition']
    filename_list = re.findall('filename=(.+)', d)
    for filename in filename_list:
        #http://docs.python-requests.org/en/master/user/quickstart/#raw-response-content
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content():
                fd.write(chunk)

The output files are now present in a filder named output.tar.gz which is located in the current working directory


## Use the below cell to get the list of jobs you submitted.

In [23]:
# get a list of jobs for user and app key, and terminalStage status

r = requests.get("%s/job/%s" % (URL,CRA_USER), auth=(CRA_USER, PASSWORD), headers=headers)
#print(r.text)

root = xml.etree.ElementTree.fromstring(r.text)
for child in root:
    if child.tag == 'jobs':
        for jobchild in child:
            if jobchild.tag == 'jobstatus':
                for statuschild in jobchild:
                    if statuschild.tag == 'selfUri':
                        for selfchild in statuschild:
                            if selfchild.tag == 'url':
                                #print(child)
                                joburi = selfchild.text
                                jobr = requests.get(joburi, auth=(CRA_USER, PASSWORD), headers=headers)
                                jobroot = xml.etree.ElementTree.fromstring(jobr.text)
                                for jobrchild in jobroot:
                                    if jobrchild.tag == 'terminalStage':
                                        jobstatus = jobrchild.text
                                        sys.stdout.write("job url: %s status terminalStage: %s\n" % (joburi,jobstatus))

job url: https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-30BF952B5E5C483E92380A7A34FC7013 status terminalStage: true
job url: https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-8995E9765E3D43378A2E5BD3A8BD54B1 status terminalStage: true
job url: https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-E8F1971F8083493585E7A97E10D2353F status terminalStage: true
job url: https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-99842FFEB3B04CBA8F132302CAED02AD status terminalStage: true
job url: https://nsgr.sdsc.edu:8443/cipresrest/v1/job/aacwb/NGBW-JOB-NEURON73_TG-DF3F830FACBE4C9B9381084A643DD93B status terminalStage: true


## Run the below cell to get more information for a specific job

In [ ]:
# get information for a single job, print out raw XML, need to set joburi according to above list

joburi = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/kenneth/NGBW-JOB-NEURON73_TG-220F7B3C7EE84BC3ADD87346E933ED5E'
r = requests.get(joburi,
                 headers= headers, auth=(CRA_USER, PASSWORD))
print(r.text)


## If you would like to delete a job, run this cell

In [8]:
# delete an old job, need to set joburi
joburi = 'https://nsgr.sdsc.edu:8443/cipresrest/v1/job/kenneth/NGBW-JOB-NEURON73_TG-7AF604008F314B43A331231E5A94C361'
r = requests.delete(joburi, auth=(CRA_USER, PASSWORD), headers=headers)
sys.stderr.write("%s\n" % r.text)